In [3]:
print(__doc__)
import time
start_time = time.time()
from copy import deepcopy, copy
import math
import scipy.io as sio
import shutil
import os
from random import shuffle
import numpy as np
from pylab import *
# from featext2 import *
import matplotlib.pyplot as plt
%matplotlib inline 
#matplotlib qt
# inline (suitable for ipython only, shown inside browser!) or qt (suitable in general, shown in external window!)
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D #, axes3d
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, ParameterGrid, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RandomizedLasso
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif, SelectFdr
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from collections import OrderedDict
import re
import datetime
import urllib
import tarfile
import joblib
from joblib import Parallel, delayed, Memory
from tempfile import mkdtemp
import copy_reg
import types
import itertools
from itertools import compress
from collections import Counter

#import multiprocessing
def _pickle_method(m):
    if m.im_self is None:
        return getattr, (m.im_class, m.im_func.func_name)
    else:
        return getattr, (m.im_self, m.im_func.func_name)
copy_reg.pickle(types.MethodType, _pickle_method)

h = .2  # step size in the mesh
names = ["NearNb", "RBFSVM1", "NaiveBayes", "MLP1", "Log.Regr", "RandFor", "AdaBoost", "EnsembleMLP"]
classifiers = [
    KNeighborsClassifier(5),
    SVC(gamma='auto', C=1),
    MLPClassifier(solver='lbfgs',alpha=1e-4,hidden_layer_sizes=(10,10),random_state=1,verbose=True),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
]

cv = StratifiedKFold(n_splits=5, random_state=42)
parameters_clf = [{'n_neighbors':range(3,10)},
              {'kernel':['rbf'], 'C':[0.01,0.1,1,10,100,1000]},
              {'solver':['lbfgs'], 'alpha':[1e-5,1e-2], 'hidden_layer_sizes':[(10,10),(50,50),(100,100)]},
              {'max_depth':[4,7,10,20],'n_estimators':[5,10,20],'max_features':[20,35,50]},
              {'solver':['lbfgs'], 'alpha':[1e-5], 'hidden_layer_sizes':[(len(names)-1,len(names)-1),(len(names)-1,2)]}
             ]
makepipe_parameters_clf = [{'classifier__'+key:p[key] for key in p} for p in parameters_clf]
makepipe_parameters_clf += [{'feature_selection__k': (750,500,100), 'feature_selection__score_func': [mutual_info_classif]},
                            {'decomp__n_components': (100,50)}]
metric = ['accuracy','f1']
dataset = 0 # all datasets (0), dataset 1-2 (1), dataset 3 (2), dataset4 (3)
download = 1 # Download pre-computed (1) data or compute them anew (0)
window = 1024
datapath = 'tmp/'

Automatically created module for IPython interactive environment


In [11]:
# trained on 1 surface tested on all
def filename(i,j,k,l):
    return 'fs_'+str(i)+'_subfs_'+str(j)+'_tr_'+str(k)+'_ts_'+str(l)

acc = np.zeros((1,4,6,6,4)) # features, subfeatures, trained, tested, (TP,TN,FN,FP)
self_acc = np.zeros((1,4,6,1,4)) # features, subfeatures, trained, cross_val_self_accuracy, (TP,TN,FN,FP)
cross_acc = np.zeros((1,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
# all_acc = np.zeros((3,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm']#,'ftfn','fnormftfn']
initial_str = """# clustered and stacked graph bogus data
=stackcluster;TP;TN;FN;FP
colors=med_blue,dark_green,yellow,red
#=sortbmarks
=nogridy
=noupperright
fontsz=3
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
initial_str2 = """# clustered and stacked graph bogus data
=stackcluster;TP;TPstd;TN;TNstd;FN;FNstd;FP;FPstd
#colors=med_blue,med_blue,med_blue,dark_green,dark_green,dark_green,yellow,yellow,yellow,red,red,red
#=sortbmarks
=nogridy
=noupperright
fontsz=3
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
respath = 'results1_old/'
for i in [0]:#in range(3):
#     if i == 0:
#         continue
    outfile = datapath+respath+feats[i]+'.perf'
    outfile1 = datapath+respath+feats[i]+'_selfaccuracy.perf'
    outfile2 = datapath+respath+feats[i]+'_crossaccuracy.perf'
#     outfile3 = 'tmpresults1_transtart/'+feats[i]+'_allaccuracy.perf'
    out = open(outfile,'w+')
    out.write(initial_str+"\n")
    out1 = open(outfile1,'w+')
    out1.write(initial_str+"\n")
    out2 = open(outfile2,'w+')
    out2.write(initial_str+"\n")
#     out3 = open(outfile3,'w+')
#     out3.write(initial_str+"\n")
    for k in range(6):
        for k2 in range(6):
            out.write("multimulti="+str(k)+"-"+str(k2)+"\n")
            for j in range(4):
                fileid = datapath+respath+filename(i,j,k,k2)+'.npz'
                print i,k,k2,j
                tmp = np.load(fileid)['cm']
                # print to outfile
                acc[i,j,k,k2,0] = round(tmp[1,1],2) # TP
                acc[i,j,k,k2,1] = round(tmp[0,0],2) # TN
                acc[i,j,k,k2,2] = 1-round(tmp[1,1],2) # FN
                acc[i,j,k,k2,3] = 1-round(tmp[0,0],2) # FP
                print 'out -->', feats[i], subfeats[j], acc[i,j,k,k2]
                out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],acc[i,j,k,k2,0],acc[i,j,k,k2,1],acc[i,j,k,k2,2],acc[i,j,k,k2,3]))
                # prepare and print to outfile1
                if k == k2:
                    if j == 0:
                        out1.write("multimulti="+str(k)+"-"+str(k2)+"\n")
                    self_acc[i,j,k,0,:] = acc[i,j,k,k2,:]
                    print 'out1-->', feats[i], subfeats[j], self_acc[i,j,k,0]
                    out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],self_acc[i,j,k,0,0],self_acc[i,j,k,0,1],self_acc[i,j,k,0,2],self_acc[i,j,k,0,3]))
                # prepare and print to outfile2
                if k != k2:                  
                    if (k < 5 and k2 == 5) or (k == 5 and k2 == 4): # all values of corresponding subfeatureset j have been filled so we can take avg and std
                        if j == 0:
                            out2.write("multimulti="+str(k)+"\n")
                        t = range(6)
                        t.remove(k)
#                         if k == 5:
                        print "DEBUG!!!: ", t, acc[i,j,k,t,:].shape
                        cross_acc[i,j,k,0,:] = np.mean(acc[i,j,k,t,:], axis=0) # avg
                        cross_acc[i,j,k,1,:] = np.std(acc[i,j,k,t,:], axis=0) # std
                        print 'out2-->', feats[i], subfeats[j], cross_acc[i,j,k,0], cross_acc[i,j,k,1]
                        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],
                                                                         cross_acc[i,j,k,0,0],
#                                                                          cross_acc[i,j,k,1,0],
                                                                         cross_acc[i,j,k,0,1],
#                                                                          cross_acc[i,j,k,1,1],
                                                                         cross_acc[i,j,k,0,2],
#                                                                          cross_acc[i,j,k,1,2],
                                                                         cross_acc[i,j,k,0,3]))
#                                                                          cross_acc[i,j,k,1,3]))
#                         out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],
#                                                                  cross_acc[i,j,k,0,0],
#                                                                  cross_acc[i,j,k,1,0],
#                                                                  cross_acc[i,j,k,0,1],
#                                                                  cross_acc[i,j,k,1,1],
#                                                                  cross_acc[i,j,k,0,2],
#                                                                  cross_acc[i,j,k,1,2],
#                                                                  cross_acc[i,j,k,0,3],
#                                                                  cross_acc[i,j,k,1,3]))
    out.write("multimulti=AVG\n")
    out1.write("multimulti=AVG\n")
    out2.write("multimulti=AVG\n")
    for j in range(4):
        avgacc = np.mean(np.mean(acc[i,j,:,:,:],axis=0),axis=0)
        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgacc[0],avgacc[1],avgacc[2],avgacc[3]))
        avgselfacc = np.mean(self_acc[i,j,:,0,:],axis=0)
        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgselfacc[0],avgselfacc[1],avgselfacc[2],avgselfacc[3]))
        avgcrossacc0 = np.mean(cross_acc[i,j,:,0,:],axis=0)
        avgcrossacc1 = np.std(cross_acc[i,j,:,0,:],axis=0)
        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc0[1],avgcrossacc0[2],avgcrossacc0[3]))
        #out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc1[0],avgcrossacc0[1],avgcrossacc1[1],avgcrossacc0[2],avgcrossacc1[2],avgcrossacc0[3],avgcrossacc1[3]))
    out.close()
    out1.close()
    out2.close()

0 0 0 0
out --> fnorm AFFT [ 0.9   0.93  0.1   0.07]
out1--> fnorm AFFT [ 0.9   0.93  0.1   0.07]
0 0 0 1
out --> fnorm FREQ [ 0.93  0.92  0.07  0.08]
out1--> fnorm FREQ [ 0.93  0.92  0.07  0.08]
0 0 0 2
out --> fnorm TIME [ 0.98  0.98  0.02  0.02]
out1--> fnorm TIME [ 0.98  0.98  0.02  0.02]
0 0 0 3
out --> fnorm BOTH [ 0.97  0.99  0.03  0.01]
out1--> fnorm BOTH [ 0.97  0.99  0.03  0.01]
0 0 1 0
out --> fnorm AFFT [ 0.38  0.98  0.62  0.02]
0 0 1 1
out --> fnorm FREQ [ 0.67  0.98  0.33  0.02]
0 0 1 2
out --> fnorm TIME [ 0.71  0.93  0.29  0.07]
0 0 1 3
out --> fnorm BOTH [ 0.76  0.94  0.24  0.06]
0 0 2 0
out --> fnorm AFFT [ 0.67  0.97  0.33  0.03]
0 0 2 1
out --> fnorm FREQ [ 0.81  0.96  0.19  0.04]
0 0 2 2
out --> fnorm TIME [ 0.86  1.    0.14  0.  ]
0 0 2 3
out --> fnorm BOTH [ 0.9   0.99  0.1   0.01]
0 0 3 0
out --> fnorm AFFT [ 0.87  0.81  0.13  0.19]
0 0 3 1
out --> fnorm FREQ [ 0.89  0.94  0.11  0.06]
0 0 3 2
out --> fnorm TIME [ 0.93  0.82  0.07  0.18]
0 0 3 3
out --> fnorm BOT

out --> fnorm FREQ [ 0.87  0.94  0.13  0.06]
DEBUG!!!:  [0, 1, 2, 3, 5] (5, 4)
out2--> fnorm FREQ [ 0.886  0.766  0.114  0.234] [ 0.01356466  0.13690873  0.01356466  0.13690873]
0 4 5 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
DEBUG!!!:  [0, 1, 2, 3, 5] (5, 4)
out2--> fnorm TIME [ 0.966  0.796  0.034  0.204] [ 0.01959592  0.1190966   0.01959592  0.1190966 ]
0 4 5 3
out --> fnorm BOTH [ 0.96  0.91  0.04  0.09]
DEBUG!!!:  [0, 1, 2, 3, 5] (5, 4)
out2--> fnorm BOTH [ 0.938  0.934  0.062  0.066] [ 0.01469694  0.03072458  0.01469694  0.03072458]
0 5 0 0
out --> fnorm AFFT [ 0.95  0.61  0.05  0.39]
0 5 0 1
out --> fnorm FREQ [ 0.92  0.68  0.08  0.32]
0 5 0 2
out --> fnorm TIME [ 1.    0.53  0.    0.47]
0 5 0 3
out --> fnorm BOTH [ 0.99  0.84  0.01  0.16]
0 5 1 0
out --> fnorm AFFT [ 0.91  0.67  0.09  0.33]
0 5 1 1
out --> fnorm FREQ [ 0.83  0.87  0.17  0.13]
0 5 1 2
out --> fnorm TIME [ 0.95  0.55  0.05  0.45]
0 5 1 3
out --> fnorm BOTH [ 0.94  0.96  0.06  0.04]
0 5 2 0
out --> fnorm AFFT

In [2]:
# trained on 2 surfaces tested on all
def filename(i,j,k1,k2,l):
    return 'fs_'+str(i)+'_subfs_'+str(j)+'_tr1_'+str(k1)+'_tr2_'+str(k2)+'_ts_'+str(l)

acc = np.zeros((3,4,6,6,6,4)) # features, subfeatures, trained1, trained2, tested, (TP,TN,FN,FP)
avg = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all tested surfaces
self_acc = np.zeros((3,4,6,6,1,4)) # features, subfeatures, trained, cross_val_self_accuracy, (TP,TN,FN,FP)
avgs = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all self tested surfaces
cross_acc = np.zeros((3,4,6,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
avgc = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> (avg,std) over all cross tested surfaces
# all_acc = np.zeros((3,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm','ftfn','fnormftfn']
initial_str = """# clustered and stacked graph bogus data
=stackcluster;TP;TN;FN;FP
colors=med_blue,dark_green,yellow,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
initial_str2 = """# clustered and stacked graph bogus data
=stackcluster;TP;TPstd;TN;TNstd;FN;FNstd;FP;FPstd
colors=med_blue,med_blue,dark_green,dark_green,yellow,yellow,red,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""

for i in [0]: #range(3):
    outfile = 'tmpresults2_transtart/'+feats[i]+'.perf'
    outfile1 = 'tmpresults2_transtart/'+feats[i]+'_selfaccuracy.perf'
    outfile2 = 'tmpresults2_transtart/'+feats[i]+'_crossaccuracy.perf'
    out = open(outfile,'w+')
    out.write(initial_str+"\n")
    out1 = open(outfile1,'w+')
    out1.write(initial_str+"\n")
    out2 = open(outfile2,'w+')
    out2.write(initial_str2+"\n")
    
    for k1 in range(6):
        for k2 in range(6):
            if k2 > k1:
                for l in range(6):
                    #if l != k1 and l != k2:
#                         if l  # print only for the first time
                    out.write("multimulti="+str(k1)+str(k2)+"-"+str(l)+"\n")
                    for j in range(4):
                        fileid = 'tmpresults2_transtart/'+filename(i,j,k1,k2,l)+'.npz'
                        print i,k1,k2,l,j
                        tmp = np.load(fileid)['cm']
                        acc[i,j,k1,k2,l,0] = round(tmp[1,1],2) # TP
                        acc[i,j,k1,k2,l,1] = round(tmp[0,0],2) # TN
                        acc[i,j,k1,k2,l,2] = 1-round(tmp[1,1],2) # FN
                        acc[i,j,k1,k2,l,3] = 1-round(tmp[0,0],2) # FP
                        avg[i,j,:] += acc[i,j,k1,k2,l,:]
                        print 'out -->', feats[i], subfeats[j], acc[i,j,k1,k2,l]
                        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],acc[i,j,k1,k2,l,0],acc[i,j,k1,k2,l,1],acc[i,j,k1,k2,l,2],acc[i,j,k1,k2,l,3]))
                        if l == k1 or l == k2: # selc accuracy
                            if j == 0 and l == k2:
                                out1.write("multimulti="+str(k1)+str(k2)+"-"+str(l)+"\n")
                            self_acc[i,j,k1,k2,0,:] = acc[i,j,k1,k2,l]
                            avgs[i,j,:] += self_acc[i,j,k1,k2,0,:]
                            print 'out1-->', feats[i], subfeats[j], self_acc[i,j,k1,k2,0]
                            if l == k2:
                                out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],self_acc[i,j,k1,k2,0,0],self_acc[i,j,k1,k2,0,1],self_acc[i,j,k1,k2,0,2],self_acc[i,j,k1,k2,0,3]))
                        if l != k1 and l != k2:
                            t = range(6)
                            t.remove(k1)
                            t.remove(k2)
                            if (l == t[-1]):
#                             if (k2 < 5 and l == 5) or (k2 == 5 and l == 4): # all values of corresponding subfeatureset j have been filled so we can take avg and std
                                if j == 0:
                                    out2.write("multimulti="+str(k1)+str(k2)+"\n")
        #                         if k == 5:
                                print "DEBUG!!!: ", t, acc[i,j,k1,k2,t,:].shape
                                cross_acc[i,j,k1,k2,0,:] = np.mean(acc[i,j,k1,k2,t,:], axis=0) # avg
                                cross_acc[i,j,k1,k2,1,:] = np.std(acc[i,j,k1,k2,t,:], axis=0) # std
                                avgc[i,j,:] += cross_acc[i,j,k1,k2,0,:]
                                print 'out2-->', feats[i], subfeats[j], cross_acc[i,j,k1,k2,0], cross_acc[i,j,k1,k2,1]
                                out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],
                                                                         cross_acc[i,j,k1,k2,0,0],
                                                                         cross_acc[i,j,k1,k2,1,0],
                                                                         cross_acc[i,j,k1,k2,0,1],
                                                                         cross_acc[i,j,k1,k2,1,1],
                                                                         cross_acc[i,j,k1,k2,0,2],
                                                                         cross_acc[i,j,k1,k2,1,2],
                                                                         cross_acc[i,j,k1,k2,0,3],
                                                                         cross_acc[i,j,k1,k2,1,3]))
    # AVG are NOT CORRECT, FIX THEM!!!!
    out.write("multimulti=AVG\n")
    out1.write("multimulti=AVG\n")
    out2.write("multimulti=AVG\n")
    avg /= 15.*6
    avgs /= 15.*2
    avgc /= 15.
    for j in range(4):
#         avgacc = np.mean(np.mean(np.mean(acc[i,j,:,:,:,:],axis=0),axis=0),axis=0)
        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avg[i,j,0],avg[i,j,1],avg[i,j,2],avg[i,j,3]))
#         avgselfacc = np.mean(np.mean(self_acc[i,j,:,:,0,:],axis=0),axis=0)
        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgs[i,j,0],avgs[i,j,1],avgs[i,j,2],avgs[i,j,3]))
        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgc[i,j,0],avgc[i,j,1],avgc[i,j,2],avgc[i,j,3]))
#         avgcrossacc0 = np.mean(np.mean(cross_acc[i,j,:,:,0,:],axis=0),axis=0)
#         avgcrossacc1 = np.std(np.mean(cross_acc[i,j,:,0,:],axis=0),axis=0)
#         out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc1[0],avgcrossacc0[1],avgcrossacc1[1],avgcrossacc0[2],avgcrossacc1[2],avgcrossacc0[3],avgcrossacc1[3]))
    out.close()
    out1.close()
    out2.close()

0 0 1 0 0
out --> fnorm AFFT [ 0.79  0.88  0.21  0.12]
out1--> fnorm AFFT [ 0.79  0.88  0.21  0.12]
0 0 1 0 1
out --> fnorm FREQ [ 0.9   0.95  0.1   0.05]
out1--> fnorm FREQ [ 0.9   0.95  0.1   0.05]
0 0 1 0 2
out --> fnorm TIME [ 0.95  0.96  0.05  0.04]
out1--> fnorm TIME [ 0.95  0.96  0.05  0.04]
0 0 1 0 3
out --> fnorm BOTH [ 0.95  0.99  0.05  0.01]
out1--> fnorm BOTH [ 0.95  0.99  0.05  0.01]
0 0 1 1 0
out --> fnorm AFFT [ 0.79  0.88  0.21  0.12]
out1--> fnorm AFFT [ 0.79  0.88  0.21  0.12]
0 0 1 1 1
out --> fnorm FREQ [ 0.9   0.95  0.1   0.05]
out1--> fnorm FREQ [ 0.9   0.95  0.1   0.05]
0 0 1 1 2
out --> fnorm TIME [ 0.95  0.96  0.05  0.04]
out1--> fnorm TIME [ 0.95  0.96  0.05  0.04]
0 0 1 1 3
out --> fnorm BOTH [ 0.95  0.99  0.05  0.01]
out1--> fnorm BOTH [ 0.95  0.99  0.05  0.01]
0 0 1 2 0
out --> fnorm AFFT [ 0.9   0.94  0.1   0.06]
0 0 1 2 1
out --> fnorm FREQ [ 0.91  0.96  0.09  0.04]
0 0 1 2 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 3
out --> fnorm BOTH [ 0.95

out --> fnorm FREQ [ 0.93  0.92  0.07  0.08]
0 0 5 3 2
out --> fnorm TIME [ 0.97  0.93  0.03  0.07]
0 0 5 3 3
out --> fnorm BOTH [ 0.94  0.99  0.06  0.01]
0 0 5 4 0
out --> fnorm AFFT [ 0.45  0.97  0.55  0.03]
DEBUG!!!:  [1, 2, 3, 4] (4, 4)
out2--> fnorm AFFT [ 0.6225  0.925   0.3775  0.075 ] [ 0.20327014  0.10136567  0.20327014  0.10136567]
0 0 5 4 1
out --> fnorm FREQ [ 0.82  0.98  0.18  0.02]
DEBUG!!!:  [1, 2, 3, 4] (4, 4)
out2--> fnorm FREQ [ 0.83    0.9725  0.17    0.0275] [ 0.06041523  0.03112475  0.06041523  0.03112475]
0 0 5 4 2
out --> fnorm TIME [ 0.92  0.99  0.08  0.01]
DEBUG!!!:  [1, 2, 3, 4] (4, 4)
out2--> fnorm TIME [ 0.91  0.96  0.09  0.04] [ 0.04062019  0.03        0.04062019  0.03      ]
0 0 5 4 3
out --> fnorm BOTH [ 0.92  0.99  0.08  0.01]
DEBUG!!!:  [1, 2, 3, 4] (4, 4)
out2--> fnorm BOTH [ 0.91   0.995  0.09   0.005] [ 0.03   0.005  0.03   0.005]
0 0 5 5 0
out --> fnorm AFFT [ 0.85  0.91  0.15  0.09]
out1--> fnorm AFFT [ 0.85  0.91  0.15  0.09]
0 0 5 5 1
out --> fno

0 1 5 1 1
out --> fnorm FREQ [ 0.92  0.91  0.08  0.09]
out1--> fnorm FREQ [ 0.92  0.91  0.08  0.09]
0 1 5 1 2
out --> fnorm TIME [ 0.96  0.92  0.04  0.08]
out1--> fnorm TIME [ 0.96  0.92  0.04  0.08]
0 1 5 1 3
out --> fnorm BOTH [ 0.94  0.99  0.06  0.01]
out1--> fnorm BOTH [ 0.94  0.99  0.06  0.01]
0 1 5 2 0
out --> fnorm AFFT [ 0.93  0.93  0.07  0.07]
0 1 5 2 1
out --> fnorm FREQ [ 0.92  0.96  0.08  0.04]
0 1 5 2 2
out --> fnorm TIME [ 0.95  0.99  0.05  0.01]
0 1 5 2 3
out --> fnorm BOTH [ 0.94  0.96  0.06  0.04]
0 1 5 3 0
out --> fnorm AFFT [ 0.98  0.71  0.02  0.29]
0 1 5 3 1
out --> fnorm FREQ [ 0.96  0.75  0.04  0.25]
0 1 5 3 2
out --> fnorm TIME [ 0.96  0.88  0.04  0.12]
0 1 5 3 3
out --> fnorm BOTH [ 0.95  0.96  0.05  0.04]
0 1 5 4 0
out --> fnorm AFFT [ 0.84  0.93  0.16  0.07]
DEBUG!!!:  [0, 2, 3, 4] (4, 4)
out2--> fnorm AFFT [ 0.93  0.84  0.07  0.16] [ 0.05522681  0.09433981  0.05522681  0.09433981]
0 1 5 4 1
out --> fnorm FREQ [ 0.91  0.96  0.09  0.04]
DEBUG!!!:  [0, 2, 3, 4] 

out --> fnorm FREQ [ 0.87  0.98  0.13  0.02]
0 3 4 2 2
out --> fnorm TIME [ 0.93  0.96  0.07  0.04]
0 3 4 2 3
out --> fnorm BOTH [ 0.94  0.98  0.06  0.02]
0 3 4 3 0
out --> fnorm AFFT [ 0.82  0.88  0.18  0.12]
out1--> fnorm AFFT [ 0.82  0.88  0.18  0.12]
0 3 4 3 1
out --> fnorm FREQ [ 0.91  0.96  0.09  0.04]
out1--> fnorm FREQ [ 0.91  0.96  0.09  0.04]
0 3 4 3 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 3 4 3 3
out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 3 4 4 0
out --> fnorm AFFT [ 0.82  0.88  0.18  0.12]
out1--> fnorm AFFT [ 0.82  0.88  0.18  0.12]
0 3 4 4 1
out --> fnorm FREQ [ 0.91  0.96  0.09  0.04]
out1--> fnorm FREQ [ 0.91  0.96  0.09  0.04]
0 3 4 4 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 3 4 4 3
out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 3 4 5 0
out --> fnorm AFFT [ 0.87  0.89  0.

In [12]:
# trained on 3 surfaces tested on all
def filename(i,j,k1,k2,k3,l):
    return 'fs_'+str(i)+'_subfs_'+str(j)+'_tr1_'+str(k1)+'_tr2_'+str(k2)+'_tr3_'+str(k3)+'_ts_'+str(l)

acc = np.zeros((3,4,6,6,6,6,4)) # features, subfeatures, trained1, trained2, trained3, tested, (TP,TN,FN,FP)
avg = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all tested surfaces
self_acc = np.zeros((3,4,6,6,6,1,4)) # features, subfeatures, trained, cross_val_self_accuracy, (TP,TN,FN,FP)
avgs = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all self tested surfaces
cross_acc = np.zeros((3,4,6,6,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
avgc = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> (avg,std) over all cross tested surfaces
# all_acc = np.zeros((3,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm','ftfn','fnormftfn']
initial_str = """# clustered and stacked graph bogus data
=stackcluster;TP;TN;FN;FP
colors=med_blue,dark_green,yellow,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
initial_str2 = """# clustered and stacked graph bogus data
=stackcluster;TP;TPstd;TN;TNstd;FN;FNstd;FP;FPstd
colors=med_blue,med_blue,dark_green,dark_green,yellow,yellow,red,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""

for i in [0]: #range(3):
    outfile = 'tmpresults3_transtart/'+feats[i]+'.perf'
    outfile1 = 'tmpresults3_transtart/'+feats[i]+'_selfaccuracy.perf'
    outfile2 = 'tmpresults3_transtart/'+feats[i]+'_crossaccuracy.perf'
    out = open(outfile,'w+')
    out.write(initial_str+"\n")
    out1 = open(outfile1,'w+')
    out1.write(initial_str+"\n")
    out2 = open(outfile2,'w+')
    out2.write(initial_str2+"\n")
    
    for k1 in range(6):
        for k2 in range(6):
            if k2 > k1:
                for k3 in range(6):
                    if k3 > k2:
                        for l in range(6):
                            #if l != k1 and l != k2:
        #                         if l  # print only for the first time
                            out.write("multimulti="+str(k1)+str(k2)+str(k3)+"-"+str(l)+"\n")
                            for j in range(4):
                                fileid = 'tmpresults3_transtart/'+filename(i,j,k1,k2,k3,l)+'.npz'
                                print i,k1,k2,k3,l,j
                                tmp = np.load(fileid)['cm']
                                acc[i,j,k1,k2,k3,l,0] = round(tmp[1,1],2) # TP
                                acc[i,j,k1,k2,k3,l,1] = round(tmp[0,0],2) # TN
                                acc[i,j,k1,k2,k3,l,2] = 1-round(tmp[1,1],2) # FN
                                acc[i,j,k1,k2,k3,l,3] = 1-round(tmp[0,0],2) # FP
                                avg[i,j,:] += acc[i,j,k1,k2,k3,l,:]
                                print 'out -->', feats[i], subfeats[j], acc[i,j,k1,k2,k3,l]
                                out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],acc[i,j,k1,k2,k3,l,0],acc[i,j,k1,k2,k3,l,1],acc[i,j,k1,k2,k3,l,2],acc[i,j,k1,k2,k3,l,3]))
                                if l == k1 or l == k2 or l == k3: # selc accuracy
                                    if j == 0 and l == k3:
                                        out1.write("multimulti="+str(k1)+str(k2)+str(k3)+"-"+str(l)+"\n")
                                    self_acc[i,j,k1,k2,k3,0,:] = acc[i,j,k1,k2,k3,l]
                                    avgs[i,j,:] += self_acc[i,j,k1,k2,k3,0,:]
                                    print 'out1-->', feats[i], subfeats[j], self_acc[i,j,k1,k2,k3,0]
                                    if l == k3:
                                        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],self_acc[i,j,k1,k2,k3,0,0],self_acc[i,j,k1,k2,k3,0,1],self_acc[i,j,k1,k2,k3,0,2],self_acc[i,j,k1,k2,k3,0,3]))
                                if l != k1 and l != k2 and l != k3:
                                    t = range(6)
                                    t.remove(k1)
                                    t.remove(k2)
                                    t.remove(k3)
                                    if (l == t[-1]):
                                    #if (k3 < 5 and l == 5) or (k3 == 5 and l == 4) or (k2 == 4 and k3 == 5 and l == 3): # all values of corresponding subfeatureset j have been filled so we can take avg and std
                                        if j == 0:
                                            out2.write("multimulti="+str(k1)+str(k2)+str(k3)+"\n")
                #                         if k == 5:
                                        print "DEBUG!!!: ", t, acc[i,j,k1,k2,k3,t,:].shape
                                        cross_acc[i,j,k1,k2,k3,0,:] = np.mean(acc[i,j,k1,k2,k3,t,:], axis=0) # avg
                                        cross_acc[i,j,k1,k2,k3,1,:] = np.std(acc[i,j,k1,k2,k3,t,:], axis=0) # std
                                        avgc[i,j,:] += cross_acc[i,j,k1,k2,k3,0,:]
                                        print 'out2-->', feats[i], subfeats[j], cross_acc[i,j,k1,k2,k3,0], cross_acc[i,j,k1,k2,k3,1]
                                        out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],
                                                                                 cross_acc[i,j,k1,k2,k3,0,0],
                                                                                 cross_acc[i,j,k1,k2,k3,1,0],
                                                                                 cross_acc[i,j,k1,k2,k3,0,1],
                                                                                 cross_acc[i,j,k1,k2,k3,1,1],
                                                                                 cross_acc[i,j,k1,k2,k3,0,2],
                                                                                 cross_acc[i,j,k1,k2,k3,1,2],
                                                                                 cross_acc[i,j,k1,k2,k3,0,3],
                                                                                 cross_acc[i,j,k1,k2,k3,1,3]))
    # AVG are NOT CORRECT, FIX THEM!!!!
    out.write("multimulti=AVG\n")
    out1.write("multimulti=AVG\n")
    out2.write("multimulti=AVG\n")
    avg /= 20.*6
    avgs /= 20.*3
    avgc /= 20.
    for j in range(4):
#         avgacc = np.mean(np.mean(np.mean(acc[i,j,:,:,:,:],axis=0),axis=0),axis=0)
        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avg[i,j,0],avg[i,j,1],avg[i,j,2],avg[i,j,3]))
#         avgselfacc = np.mean(np.mean(self_acc[i,j,:,:,0,:],axis=0),axis=0)
        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgs[i,j,0],avgs[i,j,1],avgs[i,j,2],avgs[i,j,3]))
        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgc[i,j,0],avgc[i,j,1],avgc[i,j,2],avgc[i,j,3]))
#         avgcrossacc0 = np.mean(np.mean(cross_acc[i,j,:,:,0,:],axis=0),axis=0)
#         avgcrossacc1 = np.std(np.mean(cross_acc[i,j,:,0,:],axis=0),axis=0)
#         out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc1[0],avgcrossacc0[1],avgcrossacc1[1],avgcrossacc0[2],avgcrossacc1[2],avgcrossacc0[3],avgcrossacc1[3]))
    out.close()
    out1.close()
    out2.close()

0 0 1 2 0 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 0 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 0 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 0 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 1 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 1 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 1 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 1 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 2 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 2 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91

0 0 2 5 4 2
out --> fnorm TIME [ 0.96  0.99  0.04  0.01]
DEBUG!!!:  [1, 3, 4] (3, 4)
out2--> fnorm TIME [ 0.95  0.94  0.05  0.06] [ 0.02160247  0.03559026  0.02160247  0.03559026]
0 0 2 5 4 3
out --> fnorm BOTH [ 0.95  0.99  0.05  0.01]
DEBUG!!!:  [1, 3, 4] (3, 4)
out2--> fnorm BOTH [ 0.94333333  0.99        0.05666667  0.01      ] [  9.42809042e-03   1.11022302e-16   9.42809042e-03   0.00000000e+00]
0 0 2 5 5 0
out --> fnorm AFFT [ 0.9   0.87  0.1   0.13]
out1--> fnorm AFFT [ 0.9   0.87  0.1   0.13]
0 0 2 5 5 1
out --> fnorm FREQ [ 0.92  0.94  0.08  0.06]
out1--> fnorm FREQ [ 0.92  0.94  0.08  0.06]
0 0 2 5 5 2
out --> fnorm TIME [ 0.95  0.94  0.05  0.06]
out1--> fnorm TIME [ 0.95  0.94  0.05  0.06]
0 0 2 5 5 3
out --> fnorm BOTH [ 0.96  0.96  0.04  0.04]
out1--> fnorm BOTH [ 0.96  0.96  0.04  0.04]
0 0 3 4 0 0
out --> fnorm AFFT [ 0.83  0.84  0.17  0.16]
out1--> fnorm AFFT [ 0.83  0.84  0.17  0.16]
0 0 3 4 0 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.

0 1 3 4 2 1
out --> fnorm FREQ [ 0.89  0.97  0.11  0.03]
0 1 3 4 2 2
out --> fnorm TIME [ 0.95  0.98  0.05  0.02]
0 1 3 4 2 3
out --> fnorm BOTH [ 0.95  0.96  0.05  0.04]
0 1 3 4 3 0
out --> fnorm AFFT [ 0.87  0.87  0.13  0.13]
out1--> fnorm AFFT [ 0.87  0.87  0.13  0.13]
0 1 3 4 3 1
out --> fnorm FREQ [ 0.9   0.94  0.1   0.06]
out1--> fnorm FREQ [ 0.9   0.94  0.1   0.06]
0 1 3 4 3 2
out --> fnorm TIME [ 0.95  0.93  0.05  0.07]
out1--> fnorm TIME [ 0.95  0.93  0.05  0.07]
0 1 3 4 3 3
out --> fnorm BOTH [ 0.95  0.97  0.05  0.03]
out1--> fnorm BOTH [ 0.95  0.97  0.05  0.03]
0 1 3 4 4 0
out --> fnorm AFFT [ 0.87  0.87  0.13  0.13]
out1--> fnorm AFFT [ 0.87  0.87  0.13  0.13]
0 1 3 4 4 1
out --> fnorm FREQ [ 0.9   0.94  0.1   0.06]
out1--> fnorm FREQ [ 0.9   0.94  0.1   0.06]
0 1 3 4 4 2
out --> fnorm TIME [ 0.95  0.93  0.05  0.07]
out1--> fnorm TIME [ 0.95  0.93  0.05  0.07]
0 1 3 4 4 3
out --> fnorm BOTH [ 0.95  0.97  0.05  0.03]
out1--> fnorm BOTH [ 0.95  0.97  0.05  0.03]
0 1 3 4 5 0
o

DEBUG!!!:  [0, 1, 2] (3, 4)
out2--> fnorm TIME [ 0.95666667  0.93333333  0.04333333  0.06666667] [ 0.01699673  0.04189935  0.01699673  0.04189935]
0 3 4 5 2 3
out --> fnorm BOTH [ 0.94  0.98  0.06  0.02]
DEBUG!!!:  [0, 1, 2] (3, 4)
out2--> fnorm BOTH [ 0.95666667  0.98666667  0.04333333  0.01333333] [ 0.01247219  0.00471405  0.01247219  0.00471405]
0 3 4 5 3 0
out --> fnorm AFFT [ 0.89  0.86  0.11  0.14]
out1--> fnorm AFFT [ 0.89  0.86  0.11  0.14]
0 3 4 5 3 1
out --> fnorm FREQ [ 0.92  0.93  0.08  0.07]
out1--> fnorm FREQ [ 0.92  0.93  0.08  0.07]
0 3 4 5 3 2
out --> fnorm TIME [ 0.96  0.93  0.04  0.07]
out1--> fnorm TIME [ 0.96  0.93  0.04  0.07]
0 3 4 5 3 3
out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 3 4 5 4 0
out --> fnorm AFFT [ 0.89  0.86  0.11  0.14]
out1--> fnorm AFFT [ 0.89  0.86  0.11  0.14]
0 3 4 5 4 1
out --> fnorm FREQ [ 0.92  0.93  0.08  0.07]
out1--> fnorm FREQ [ 0.92  0.93  0.08  0.07]
0 3 4 5 4 2
out --> fnorm TIME [ 0.96

In [14]:
# trained on 4 surfaces tested on all
def filename(i,j,k1,k2,k3,k4,l):
    return 'fs_'+str(i)+'_subfs_'+str(j)+'_tr1_'+str(k1)+'_tr2_'+str(k2)+'_tr3_'+str(k3)+'_tr4_'+str(k4)+'_ts_'+str(l)

acc = np.zeros((3,4,6,6,6,6,6,4)) # features, subfeatures, trained1, trained2, trained3, trained4, tested, (TP,TN,FN,FP)
avg = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all tested surfaces
self_acc = np.zeros((3,4,6,6,6,6,1,4)) # features, subfeatures, trained, cross_val_self_accuracy, (TP,TN,FN,FP)
avgs = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all self tested surfaces
cross_acc = np.zeros((3,4,6,6,6,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
avgc = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> (avg,std) over all cross tested surfaces
# all_acc = np.zeros((3,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm','ftfn','fnormftfn']
initial_str = """# clustered and stacked graph bogus data
=stackcluster;TP;TN;FN;FP
colors=med_blue,dark_green,yellow,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
initial_str2 = """# clustered and stacked graph bogus data
=stackcluster;TP;TPstd;TN;TNstd;FN;FNstd;FP;FPstd
colors=med_blue,med_blue,dark_green,dark_green,yellow,yellow,red,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""

for i in [0]: #range(3):
    outfile = 'tmpresults4_transtart/'+feats[i]+'.perf'
    outfile1 = 'tmpresults4_transtart/'+feats[i]+'_selfaccuracy.perf'
    outfile2 = 'tmpresults4_transtart/'+feats[i]+'_crossaccuracy.perf'
    out = open(outfile,'w+')
    out.write(initial_str+"\n")
    out1 = open(outfile1,'w+')
    out1.write(initial_str+"\n")
    out2 = open(outfile2,'w+')
    out2.write(initial_str2+"\n")
    
    for k1 in range(6):
        for k2 in range(6):
            if k2 > k1:
                for k3 in range(6):
                    if k3 > k2:
                        for k4 in range(6):
                            if k4 > k3:
                                for l in range(6):
                                    #if l != k1 and l != k2:
                #                         if l  # print only for the first time
                                    out.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+"-"+str(l)+"\n")
                                    for j in range(4):
                                        fileid = 'tmpresults4_transtart/'+filename(i,j,k1,k2,k3,k4,l)+'.npz'
                                        print i,k1,k2,k3,k4,l,j
                                        tmp = np.load(fileid)['cm']
                                        acc[i,j,k1,k2,k3,k4,l,0] = round(tmp[1,1],2) # TP
                                        acc[i,j,k1,k2,k3,k4,l,1] = round(tmp[0,0],2) # TN
                                        acc[i,j,k1,k2,k3,k4,l,2] = 1-round(tmp[1,1],2) # FN
                                        acc[i,j,k1,k2,k3,k4,l,3] = 1-round(tmp[0,0],2) # FP
                                        avg[i,j,:] += acc[i,j,k1,k2,k3,k4,l,:]
                                        print 'out -->', feats[i], subfeats[j], acc[i,j,k1,k2,k3,k4,l]
                                        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],acc[i,j,k1,k2,k3,k4,l,0],acc[i,j,k1,k2,k3,k4,l,1],acc[i,j,k1,k2,k3,k4,l,2],acc[i,j,k1,k2,k3,k4,l,3]))
                                        if l == k1 or l == k2 or l == k3 or l == k4: # selc accuracy
                                            if j == 0 and l == k4:
                                                out1.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+"-"+str(l)+"\n")
                                            self_acc[i,j,k1,k2,k3,k4,0,:] = acc[i,j,k1,k2,k3,k4,l]
                                            avgs[i,j,:] += self_acc[i,j,k1,k2,k3,k4,0,:]
                                            print 'out1-->', feats[i], subfeats[j], self_acc[i,j,k1,k2,k3,k4,0]
                                            if l == k4:
                                                out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],self_acc[i,j,k1,k2,k3,k4,0,0],self_acc[i,j,k1,k2,k3,k4,0,1],self_acc[i,j,k1,k2,k3,k4,0,2],self_acc[i,j,k1,k2,k3,k4,0,3]))
                                        if l != k1 and l != k2 and l != k3 and l!= k4:
                                            t = range(6)
                                            t.remove(k1)
                                            t.remove(k2)
                                            t.remove(k3)
                                            t.remove(k4)
                                            if (l == t[-1]):
                                            #if (k3 < 5 and l == 5) or (k3 == 5 and l == 4) or (k2 == 4 and k3 == 5 and l == 3): # all values of corresponding subfeatureset j have been filled so we can take avg and std
                                                if j == 0:
                                                    out2.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+"\n")
                        #                         if k == 5:
                                                print "DEBUG!!!: ", t, acc[i,j,k1,k2,k3,k4,t,:].shape
                                                cross_acc[i,j,k1,k2,k3,k4,0,:] = np.mean(acc[i,j,k1,k2,k3,k4,t,:], axis=0) # avg
                                                cross_acc[i,j,k1,k2,k3,k4,1,:] = np.std(acc[i,j,k1,k2,k3,k4,t,:], axis=0) # std
                                                avgc[i,j,:] += cross_acc[i,j,k1,k2,k3,k4,0,:]
                                                print 'out2-->', feats[i], subfeats[j], cross_acc[i,j,k1,k2,k3,k4,0], cross_acc[i,j,k1,k2,k3,k4,1]
                                                out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,0,0],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,1,0],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,0,1],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,1,1],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,0,2],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,1,2],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,0,3],
                                                                                         cross_acc[i,j,k1,k2,k3,k4,1,3]))
    # AVG are NOT CORRECT, FIX THEM!!!!
    out.write("multimulti=AVG\n")
    out1.write("multimulti=AVG\n")
    out2.write("multimulti=AVG\n")
    avg /= 15.*6
    avgs /= 15.*4
    avgc /= 15.
    for j in range(4):
#         avgacc = np.mean(np.mean(np.mean(acc[i,j,:,:,:,:],axis=0),axis=0),axis=0)
        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avg[i,j,0],avg[i,j,1],avg[i,j,2],avg[i,j,3]))
#         avgselfacc = np.mean(np.mean(self_acc[i,j,:,:,0,:],axis=0),axis=0)
        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgs[i,j,0],avgs[i,j,1],avgs[i,j,2],avgs[i,j,3]))
        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgc[i,j,0],avgc[i,j,1],avgc[i,j,2],avgc[i,j,3]))
#         avgcrossacc0 = np.mean(np.mean(cross_acc[i,j,:,:,0,:],axis=0),axis=0)
#         avgcrossacc1 = np.std(np.mean(cross_acc[i,j,:,0,:],axis=0),axis=0)
#         out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc1[0],avgcrossacc0[1],avgcrossacc1[1],avgcrossacc0[2],avgcrossacc1[2],avgcrossacc0[3],avgcrossacc1[3]))
    out.close()
    out1.close()
    out2.close()

0 0 1 2 3 0 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 0 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 3 0 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 3 0 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 3 1 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 1 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 3 1 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 3 1 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 3 2 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 2 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1-

out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 2 3 4 1 0
out --> fnorm AFFT [ 0.68  0.96  0.32  0.04]
0 0 2 3 4 1 1
out --> fnorm FREQ [ 0.87  0.95  0.13  0.05]
0 0 2 3 4 1 2
out --> fnorm TIME [ 0.93  0.92  0.07  0.08]
0 0 2 3 4 1 3
out --> fnorm BOTH [ 0.95  0.99  0.05  0.01]
0 0 2 3 4 2 0
out --> fnorm AFFT [ 0.88  0.86  0.12  0.14]
out1--> fnorm AFFT [ 0.88  0.86  0.12  0.14]
0 0 2 3 4 2 1
out --> fnorm FREQ [ 0.92  0.95  0.08  0.05]
out1--> fnorm FREQ [ 0.92  0.95  0.08  0.05]
0 0 2 3 4 2 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 2 3 4 2 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 2 3 4 3 0
out --> fnorm AFFT [ 0.88  0.86  0.12  0.14]
out1--> fnorm AFFT [ 0.88  0.86  0.12  0.14]
0 0 2 3 4 3 1
out --> fnorm FREQ [ 0.92  0.95  0.08  0.05]
out1--> fnorm FREQ [ 0.92  0.95  0.08  0.05]
0 0 2 3 4 3 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.0

out --> fnorm TIME [ 0.96  0.94  0.04  0.06]
out1--> fnorm TIME [ 0.96  0.94  0.04  0.06]
0 1 2 4 5 2 3
out --> fnorm BOTH [ 0.95  0.98  0.05  0.02]
out1--> fnorm BOTH [ 0.95  0.98  0.05  0.02]
0 1 2 4 5 3 0
out --> fnorm AFFT [ 0.97  0.74  0.03  0.26]
DEBUG!!!:  [0, 3] (2, 4)
out2--> fnorm AFFT [ 0.975  0.755  0.025  0.245] [ 0.005  0.015  0.005  0.015]
0 1 2 4 5 3 1
out --> fnorm FREQ [ 0.98  0.74  0.02  0.26]
DEBUG!!!:  [0, 3] (2, 4)
out2--> fnorm FREQ [ 0.975  0.815  0.025  0.185] [ 0.005  0.075  0.005  0.075]
0 1 2 4 5 3 2
out --> fnorm TIME [ 0.97  0.93  0.03  0.07]
DEBUG!!!:  [0, 3] (2, 4)
out2--> fnorm TIME [ 0.975  0.94   0.025  0.06 ] [ 0.005  0.01   0.005  0.01 ]
0 1 2 4 5 3 3
out --> fnorm BOTH [ 0.96  0.93  0.04  0.07]
DEBUG!!!:  [0, 3] (2, 4)
out2--> fnorm BOTH [ 0.97   0.955  0.03   0.045] [ 0.01   0.025  0.01   0.025]
0 1 2 4 5 4 0
out --> fnorm AFFT [ 0.93  0.9   0.07  0.1 ]
out1--> fnorm AFFT [ 0.93  0.9   0.07  0.1 ]
0 1 2 4 5 4 1
out --> fnorm FREQ [ 0.93  0.93  0.0

In [16]:
# trained on 5 surfaces tested on all
def filename(i,j,k1,k2,k3,k4,k5,l):
    return 'fs_'+str(i)+'_subfs_'+str(j)+'_tr1_'+str(k1)+'_tr2_'+str(k2)+'_tr3_'+str(k3)+'_tr4_'+str(k4)+'_tr5_'+str(k5)+'_ts_'+str(l)

acc = np.zeros((3,4,6,6,6,6,6,6,4)) # features, subfeatures, trained1, trained2, trained3, trained4, trained5, tested, (TP,TN,FN,FP)
avg = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all tested surfaces
self_acc = np.zeros((3,4,6,6,6,6,6,1,4)) # features, subfeatures, trained, cross_val_self_accuracy, (TP,TN,FN,FP)
avgs = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> avg over all self tested surfaces
cross_acc = np.zeros((3,4,6,6,6,6,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
avgc = np.zeros((3,4,4)) # features, subfeatures, (TP,TN,FN,FP) -> (avg,std) over all cross tested surfaces
# all_acc = np.zeros((3,4,6,2,4)) # features, subfeatures, trained, (tested avg, tested std), (TP,TN,FN,FP)
subfeats = ['AFFT','FREQ','TIME','BOTH']
feats = ['fnorm','ftfn','fnormftfn']
initial_str = """# clustered and stacked graph bogus data
=stackcluster;TP;TN;FN;FP
colors=med_blue,dark_green,yellow,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""
initial_str2 = """# clustered and stacked graph bogus data
=stackcluster;TP;TPstd;TN;TNstd;FN;FNstd;FP;FPstd
colors=med_blue,med_blue,dark_green,dark_green,yellow,yellow,red,red
#=sortbmarks
=nogridy
=noupperright
fontsz=2
legendx=right
legendy=center
datascale=50
yformat=%g%%
xlabel=TrainedON-TestedON
ylabel=Metrics

=table"""

for i in [0]: #range(3):
    outfile = 'tmpresults5_transtart/'+feats[i]+'.perf'
    outfile1 = 'tmpresults5_transtart/'+feats[i]+'_selfaccuracy.perf'
    outfile2 = 'tmpresults5_transtart/'+feats[i]+'_crossaccuracy.perf'
    out = open(outfile,'w+')
    out.write(initial_str+"\n")
    out1 = open(outfile1,'w+')
    out1.write(initial_str+"\n")
    out2 = open(outfile2,'w+')
    out2.write(initial_str2+"\n")
    
    for k1 in range(6):
        for k2 in range(6):
            if k2 > k1:
                for k3 in range(6):
                    if k3 > k2:
                        for k4 in range(6):
                            if k4 > k3:
                                for k5 in range(6):
                                    if k5 > k4:
                                        for l in range(6):
                                            #if l != k1 and l != k2:
                        #                         if l  # print only for the first time
                                            out.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+str(k5)+"-"+str(l)+"\n")
                                            for j in range(4):
                                                fileid = 'tmpresults5_transtart/'+filename(i,j,k1,k2,k3,k4,k5,l)+'.npz'
                                                print i,k1,k2,k3,k4,k5,l,j
                                                tmp = np.load(fileid)['cm']
                                                acc[i,j,k1,k2,k3,k4,k5,l,0] = round(tmp[1,1],2) # TP
                                                acc[i,j,k1,k2,k3,k4,k5,l,1] = round(tmp[0,0],2) # TN
                                                acc[i,j,k1,k2,k3,k4,k5,l,2] = 1-round(tmp[1,1],2) # FN
                                                acc[i,j,k1,k2,k3,k4,k5,l,3] = 1-round(tmp[0,0],2) # FP
                                                avg[i,j,:] += acc[i,j,k1,k2,k3,k4,k5,l,:]
                                                print 'out -->', feats[i], subfeats[j], acc[i,j,k1,k2,k3,k4,k5,l]
                                                out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],acc[i,j,k1,k2,k3,k4,k5,l,0],acc[i,j,k1,k2,k3,k4,k5,l,1],acc[i,j,k1,k2,k3,k4,k5,l,2],acc[i,j,k1,k2,k3,k4,k5,l,3]))
                                                if l == k1 or l == k2 or l == k3 or l == k4 or l == k5: # selc accuracy
                                                    if j == 0 and l == k5:
                                                        out1.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+str(k5)+"-"+str(l)+"\n")
                                                    self_acc[i,j,k1,k2,k3,k4,k5,0,:] = acc[i,j,k1,k2,k3,k4,k5,l]
                                                    avgs[i,j,:] += self_acc[i,j,k1,k2,k3,k4,k5,0,:]
                                                    print 'out1-->', feats[i], subfeats[j], self_acc[i,j,k1,k2,k3,k4,k5,0]
                                                    if l == k5:
                                                        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],self_acc[i,j,k1,k2,k3,k4,k5,0,0],self_acc[i,j,k1,k2,k3,k4,k5,0,1],self_acc[i,j,k1,k2,k3,k4,k5,0,2],self_acc[i,j,k1,k2,k3,k4,k5,0,3]))
                                                if l != k1 and l != k2 and l != k3 and l!= k4 and l!= k5:
                                                    t = range(6)
                                                    t.remove(k1)
                                                    t.remove(k2)
                                                    t.remove(k3)
                                                    t.remove(k4)
                                                    t.remove(k5)
                                                    if (l == t[-1]):
                                                    #if (k3 < 5 and l == 5) or (k3 == 5 and l == 4) or (k2 == 4 and k3 == 5 and l == 3): # all values of corresponding subfeatureset j have been filled so we can take avg and std
                                                        if j == 0:
                                                            out2.write("multimulti="+str(k1)+str(k2)+str(k3)+str(k4)+str(k5)+"\n")
                                #                         if k == 5:
                                                        print "DEBUG!!!: ", t, acc[i,j,k1,k2,k3,k4,k5,t,:].shape
                                                        cross_acc[i,j,k1,k2,k3,k4,k5,0,:] = np.mean(acc[i,j,k1,k2,k3,k4,k5,t,:], axis=0) # avg
                                                        cross_acc[i,j,k1,k2,k3,k4,k5,1,:] = np.std(acc[i,j,k1,k2,k3,k4,k5,t,:], axis=0) # std
                                                        avgc[i,j,:] += cross_acc[i,j,k1,k2,k3,k4,k5,0,:]
                                                        print 'out2-->', feats[i], subfeats[j], cross_acc[i,j,k1,k2,k3,k4,k5,0], cross_acc[i,j,k1,k2,k3,k4,k5,1]
                                                        out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,0,0],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,1,0],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,0,1],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,1,1],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,0,2],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,1,2],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,0,3],
                                                                                                 cross_acc[i,j,k1,k2,k3,k4,k5,1,3]))
    # AVG are NOT CORRECT, FIX THEM!!!!
    out.write("multimulti=AVG\n")
    out1.write("multimulti=AVG\n")
    out2.write("multimulti=AVG\n")
    avg /= 6.*6
    avgs /= 6.*5
    avgc /= 6.
    for j in range(4):
#         avgacc = np.mean(np.mean(np.mean(acc[i,j,:,:,:,:],axis=0),axis=0),axis=0)
        out.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avg[i,j,0],avg[i,j,1],avg[i,j,2],avg[i,j,3]))
#         avgselfacc = np.mean(np.mean(self_acc[i,j,:,:,0,:],axis=0),axis=0)
        out1.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgs[i,j,0],avgs[i,j,1],avgs[i,j,2],avgs[i,j,3]))
        out2.write("%s %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgc[i,j,0],avgc[i,j,1],avgc[i,j,2],avgc[i,j,3]))
#         avgcrossacc0 = np.mean(np.mean(cross_acc[i,j,:,:,0,:],axis=0),axis=0)
#         avgcrossacc1 = np.std(np.mean(cross_acc[i,j,:,0,:],axis=0),axis=0)
#         out2.write("%s %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f\n" % (subfeats[j],avgcrossacc0[0],avgcrossacc1[0],avgcrossacc0[1],avgcrossacc1[1],avgcrossacc0[2],avgcrossacc1[2],avgcrossacc0[3],avgcrossacc1[3]))
    out.close()
    out1.close()
    out2.close()

0 0 1 2 3 4 0 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 4 0 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 3 4 0 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 3 4 0 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 3 4 1 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 4 1 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 2 3 4 1 2
out --> fnorm TIME [ 0.96  0.97  0.04  0.03]
out1--> fnorm TIME [ 0.96  0.97  0.04  0.03]
0 0 1 2 3 4 1 3
out --> fnorm BOTH [ 0.97  0.98  0.03  0.02]
out1--> fnorm BOTH [ 0.97  0.98  0.03  0.02]
0 0 1 2 3 4 2 0
out --> fnorm AFFT [ 0.88  0.89  0.12  0.11]
out1--> fnorm AFFT [ 0.88  0.89  0.12  0.11]
0 0 1 2 3 4 2 1
out --> fnorm FREQ [ 0.91  0.9

out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 0 1 3 4 5 4 0
out --> fnorm AFFT [ 0.82  0.87  0.18  0.13]
out1--> fnorm AFFT [ 0.82  0.87  0.18  0.13]
0 0 1 3 4 5 4 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 3 4 5 4 2
out --> fnorm TIME [ 0.94  0.96  0.06  0.04]
out1--> fnorm TIME [ 0.94  0.96  0.06  0.04]
0 0 1 3 4 5 4 3
out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 0 1 3 4 5 5 0
out --> fnorm AFFT [ 0.82  0.87  0.18  0.13]
out1--> fnorm AFFT [ 0.82  0.87  0.18  0.13]
0 0 1 3 4 5 5 1
out --> fnorm FREQ [ 0.91  0.95  0.09  0.05]
out1--> fnorm FREQ [ 0.91  0.95  0.09  0.05]
0 0 1 3 4 5 5 2
out --> fnorm TIME [ 0.94  0.96  0.06  0.04]
out1--> fnorm TIME [ 0.94  0.96  0.06  0.04]
0 0 1 3 4 5 5 3
out --> fnorm BOTH [ 0.96  0.98  0.04  0.02]
out1--> fnorm BOTH [ 0.96  0.98  0.04  0.02]
0 0 2 3 4 5 0 0
out --> fnorm AFFT [ 0.87  0.87  0.13  0.13]
o